In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
import os 
import time
import pandas as pd
import numpy as np
import re
import unicodedata
import pickle


日経メディカルの情報は最新すぎる

In [10]:
os.getcwd()
os.chdir("C:/Users/hakos/GDrive/Downloads/Research/data/yakka")
data=pd.read_csv('../merged/len3_ndb_blp_DN_firm.csv',encoding='utf-8',index_col=0)
data.columns

C:\Users\hakos\AppData\Local\Temp\ipykernel_38536\3610183173.py:3: DtypeWarning: Columns (118,119,131) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('../merged/len3_ndb_blp_DN_firm.csv',encoding='utf-8',index_col=0)


Index(['薬効分類', '薬効分類名称', '医薬品コード', '医薬品名', '薬価基準収載医薬品コード', '薬価_x', '後発品区分',
       '総計', 'year', 'in_hospital',
       ...
       '同一剤形・規格の後発医薬品がある先発医薬品', '薬価_y', '経過措置による使用期限', '備考', 'Unnamed: 15',
       'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 4', '_merge'],
      dtype='object', length=132)

iyakuinfo, pdmaは先にmergeする

In [11]:

# with open('name2cleaned.pkl', 'rb') as f:
#     name2cleaned = pickle.load(f)
# with open('cleaned2firm.pkl', 'rb') as f:
#     cleaned2firm = pickle.load(f)
# with open('name2firm.pkl', 'rb') as f:
#     name2firm = pickle.load(f)
# with open('name2firm_ii.pkl', 'rb') as f:
#     name2firm_ii = pickle.load(f)
# with open('name2firm_ma.pkl', 'rb') as f:
#     name2firm_ma = pickle.load(f)
# with open('name2firm_nk.pkl', 'rb') as f:
#     name2firm_nk = pickle.load(f)

# data.loc[data["メーカー名"].isna(),"メーカー名"]=data.loc[data["メーカー名"].isna(),"医薬品名"].map(name2firm)
# data.loc[data["メーカー名"].isna(),"メーカー名"]=data.loc[data["メーカー名"].isna(),"医薬品名"].map(name2firm_ii)
# data.loc[data["メーカー名"].isna(),"メーカー名"]=data.loc[data["メーカー名"].isna(),"医薬品名"].map(name2firm_ma)
# data.loc[data["メーカー名"].isna(),"メーカー名"]=data.loc[data["メーカー名"].isna(),"医薬品名"].map(name2cleaned).map(cleaned2firm)
# data.loc[data["メーカー名"].isna(),"メーカー名"]=data.loc[data["メーカー名"].isna(),"医薬品名"].map(name2firm_nk)
# data_lack=data[data["メーカー名"].isna()]
# data_lack["医薬品名"].value_counts(),data_lack["薬価基準収載医薬品コード"].value_counts()

In [12]:
len(set(data[data["メーカー名"].isna()]["医薬品名"]))
data[data["メーカー名"].isna()]["医薬品名"].value_counts()
med_name=list(set(data[data["メーカー名"].isna()]["医薬品名"]))
len(med_name)

298

In [13]:
# make name clean
# クリーン前のテキストをキー、クリーン後のテキストを値とする辞書を作成
name2cleaned = {}
cleaned=[]
# テキストごとに処理を行うループ
for name in med_name:
    # クリーン処理を実行
    cleaned_name = unicodedata.normalize('NFKC', re.sub(r'\u3000', ' ', re.search(r'.*\u3000', name).group()) if re.search(r'.*\u3000', name) else name)
    
    cleaned.append(cleaned_name)
    # クリーン前のテキストをキー、クリーン後のテキストを値として辞書に追加
    name2cleaned[name] = cleaned_name

# 辞書を出力
# for original_name in name2cleaned.keys():
#     print(f'Original name: {original_name}')
#     print(f'Cleaned name:  {name2cleaned[original_name]}')
#     print()


## test

In [14]:

# os.chdir("C:\\Users\\hakos\\Downloads")
# os.getcwd()
# key="アレグラ"
# driver = webdriver.Firefox()
# driver.minimize_window()
# # ウェブページを開く
# driver.get(f'https://medical.nikkeibp.co.jp/inc/all/drugdic/search?words={key}')

# wait = WebDriverWait(driver, 1)
# element = wait.until(EC.presence_of_element_located((By.XPATH,"//*[@id='article01']/ul/li/div/a")))
# # element = driver.find_element(By.XPATH,"//*[@id='article01']/ul/li/div/a")

# # HTML要素からURLを取得します
# url = element.get_attribute("href")

# # 取得したURLにアクセスします
# driver.get(url)

# try:
#     element = wait.until(EC.presence_of_element_located((By.XPATH,"//*[@id='id-body']/div[1]/div[2]/div[1]/div/div[3]/div[2]/dl[2]/dd/div/a/span")))
#     maker = element.text
#     try:
#         element = driver.find_element(By.XPATH,"/html/body/div[1]/div[1]/ul/li[last()]/a")
#         name = element.text
#     except:
#         name = None
# except:
#     maker = None

# print(name,maker)

# driver.quit()

アレグラ錠60mg サノフィ


動的サイトなのでbs4じゃ無理らしい

In [92]:
#　初期化　危ない
clean=0
name2firm={}
name2name={}

In [93]:
os.chdir("C:\\Users\\hakos\\Downloads")
os.getcwd()
def get_maker(key):
    driver = webdriver.Firefox()
    driver.minimize_window()
    # ウェブページを開く
    driver.get(f'https://medical.nikkeibp.co.jp/inc/all/drugdic/search?words={key}')

    wait = WebDriverWait(driver, 5)
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH,"//*[@id='article01']/ul/li/div/a")))
        # element = driver.find_element(By.XPATH,"//*[@id='article01']/ul/li/div/a")
    except:
        driver.quit()
        return (None,None)

    # HTML要素からURLを取得します
    url = element.get_attribute("href")

    # 取得したURLにアクセスします
    driver.get(url)

    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH,"//*[@id='id-body']/div[1]/div[2]/div[1]/div/div[3]/div[2]/dl[2]/dd/div/a/span")))
        maker = element.text
        try:
            element = driver.find_element(By.XPATH,"/html/body/div[1]/div[1]/ul/li[last()]/a")
            name = element.text
        except:
            name = None
    except:
        maker = None

    driver.quit()
    return (maker,name)
none_count=0
# med_name=list(set(data[data["メーカー名"].isna()]["医薬品名"]))
failed=[]
if clean:
    med_name=cleaned
for i,key in enumerate(med_name):
    if key not in name2firm.keys():
        maker,name=get_maker(key)
        if maker is None:
            none_count+=1
            failed.append(key)
        else:
            name2firm[key]=maker
            name2name[key]=name
        # print(maker,name)
    else:
        continue
    print(i,end=" ")
    time.sleep(1)

0 1 2 3 4 5 6 7 

In [94]:
name2firm

{'ベギータ坐剤50 ': 'シオノケミカル',
 'エプカロール錠50μg ': '東和薬品',
 'ハドドリン錠「10」 ': '辰巳化学',
 'マーヨン錠「25μg」 ': '辰巳化学',
 'プロカテロール塩酸塩錠25μg「テバ」 ': '武田テバ薬品',
 'エプカロール錠25μg ': '東和薬品',
 'アデフロニックズポ50 ': '武田テバファーマ',
 'マーヨン錠「50μg」 ': '辰巳化学'}

In [95]:
import pickle
os.chdir("C:/Users/hakos/GDrive/Downloads/Research/data/yakka")

if not clean:
    file_path = 'name2firm_nk.pkl'
    with open(file_path, 'wb') as file:
        pickle.dump(name2firm, file)
    file_path = 'name2name_nk.pkl'
    with open(file_path, 'wb') as file:
        pickle.dump(name2name, file)
else:
    file_path = 'cleaned2firm_nk.pkl'
    with open(file_path, 'wb') as file:
        pickle.dump(name2firm, file)    
    file_path = 'name2cleaned_nk.pkl'
    with open(file_path, 'wb') as file:
        pickle.dump(name2cleaned, file)

In [96]:
# 残り全体
if clean:
    data.loc[data["メーカー名"].isna(),"メーカー名"]=data.loc[data["メーカー名"].isna(),"医薬品名"].map(name2cleaned).map(name2firm)
else:
    data.loc[data["メーカー名"].isna(),"メーカー名"]=data.loc[data["メーカー名"].isna(),"医薬品名"].map(name2firm)
data[data["メーカー名"].isna()].shape

(0, 132)

In [97]:
# 残り医薬品名
len(set(data[data["メーカー名"].isna()]["医薬品名"]))

0

In [98]:
set(data[data["メーカー名"].isna()]["医薬品名"])

set()

手作業

In [78]:
name2firm['アルデイン坐剤２０ｍｇ']="長生堂製薬"
name2firm['アンブロキソール塩酸塩錠１５ｍｇ「タナベ」']="長生堂製薬"
name2firm['イオウ・サリチル酸・チアントール軟膏']="日本薬品工業"
name2firm['ウブテック錠５ｍｇ']="大正薬品工業"
name2firm['エピナスチン塩酸塩錠２０ｍｇ「タイヨー」']="武田テバ"
name2firm['オリベート錠２\u3000２ｍｇ']="沢井製薬"
name2firm['クラレットカプセル１ｍｇ']="龍角散"
name2firm['クラレット細粒０．１％']="龍角散"
name2firm['クロトリマゾール１００ｍｇ腟錠']="富士製薬"
name2firm['サワダロン錠２００ｍｇ']="沢井"
name2firm['ジクロフェナクナトリウム坐剤５０ｍｇ「タナベ」']="長生堂製薬"
name2firm['ソロムコ錠１５ｍｇ']="共和薬品工業"
name2firm['タイペミンカプセル１ｍｇ']="日本ジェネリック"
name2firm['チクロピジン塩酸塩錠１００ｍｇ「タイヨー」']="武田テバファーマ"
name2firm['ニチステート錠１００ｍｇ']="日医工"
name2firm['ビホナゾールクリーム１％「タイヨー」']="武田テバ"
name2firm['ピロキシカム２０ｍｇ坐剤']="長生堂製薬"
name2firm['フルチカゾンプロピオン酸エステル日本臓器１１２\u3000８．１６ｍｇ１６']="日本臓器製薬"
name2firm['フルチカゾンプロピオン酸エステル５０トーワ２８\u3000２．０４ｍｇ４']="東和薬品"
name2firm['フルチカゾンプロピオン酸エステル５０日医工２８\u3000２．０４ｍｇ４']="日医工"
name2firm['フルチカゾンプロピオン酸エステル５０日本臓器２８\u3000２．０４ｍｇ４']="日本臓器製薬"
name2firm['フルチカゾンプロピオン酸エステル５０ＣＥＯ２８\u3000２．０４ｍｇ４']="セオリアファーマ"
name2firm['フルチカゾンプロピオン酸エステル５０ＤＳＰ２８\u3000２．０４ｍｇ４']="東興薬品"
name2firm['フルチカゾンプロピオン酸エステル５０ＪＧ２８\u3000２．０４ｍｇ４']="長生堂製薬"
name2firm['フルチカゾンプロピオン酸エステル５０ＪＧ５６\u3000４．０８ｍｇ８']="長生堂製薬"
name2firm['フルチカゾン点鼻液５０μｇイセイ２８噴霧用\u3000２．０４ｍｇ４ｍＬ']="コーアイセイ"
name2firm['フルチカゾン点鼻５０μｇ「アメル」２８噴霧\u3000２．０４ｍｇ４ｍＬ']="共和薬品"
name2firm['フルチカゾン点鼻５０μｇ「イセイ」５６噴霧\u3000４．０８ｍｇ８ｍＬ']="コーアイセイ"
name2firm['フルチカゾン点鼻５０μｇ「サワイ」２８噴霧\u3000２．０４ｍｇ４ｍＬ']="沢井製薬"
name2firm['フルチカゾン点鼻５０μｇ「トーワ」２８噴霧\u3000２．０４ｍｇ４ｍＬ']="東和薬品"
name2firm['フルチカゾン点鼻５０μｇ「ＮｉｋＰ」２８噴霧\u3000２．０４ｍｇ４ｍＬ']="日医工ファーマ"
name2firm['プロピオン酸フルチカゾン点鼻５０ＣＨ２８\u3000２．０４ｍｇ４ｍＬ']="長生堂製薬"
name2firm['プロピオン酸フルチカゾン点鼻５０ＣＨ５６\u3000４．０８ｍｇ８ｍＬ']="長生堂製薬"
name2firm['ベクロメタゾンパウダー２５μｇトーワ\u3000１．５０ｍｇ０．９０８７ｇ']="東和薬品"
name2firm['ベクロメタゾン点鼻液５０μｇファイザー\u3000９．３７５ｍｇ７．５ｇ']="マイラン製薬"
name2firm['ベタメタゾン酪酸エステルプロピオン酸クリーム０．０５％「ＭＹＫ」']="前田薬品工業"
name2firm['ベタメタゾン酪酸エステルプロピオン酸ローション０．０５％ＭＹＫ']="前田薬品工業"
name2firm['ベーエム錠１００ｍｇ']="キョーリンリメディオ"
name2firm['ホスポールクリーム１％']="沢井製薬"
name2firm['ボナフェック坐剤５０\u3000５０ｍｇ']="日新製薬"
name2firm['ミロピンカプセル１ｍｇ']="沢井製薬"
name2firm['メチレフト錠２５\u3000０．０２５ｍｇ']="沢井製薬"
name2firm['メチレフト錠５０\u3000０．０５ｍｇ']="沢井製薬"
name2firm['ルアダン錠２００ｍｇ']="科研製薬"
name2firm['ロキソプロフェンナトリウム錠６０ｍｇ「タイヨー」']="武田テバファーマ"
name2firm['ロンバニンカプセル１ｍｇ']="辰巳化学"
name2firm['Ｄソルビトール内用液６５％「マルイシ」']="丸石製薬"
name2firm['Ｄソルビトール原末「マルイシ」']="丸石製薬"
name2firm['Ｄソルビトール液\u3000６５％']="丸石製薬"
name2firm['Ｄソルビトール液\u3000７５％']="丸石製薬"
name2firm['Ｄソルビトール経口液７５％「コーワ」']="興和"

In [79]:
# 残り全体
# data.loc[data["メーカー名"].isna(),"メーカー名"]=data.loc[data["メーカー名"].isna(),"医薬品名"].map(name2cleaned).map(name2firm)
data.loc[data["メーカー名"].isna(),"メーカー名"]=data.loc[data["メーカー名"].isna(),"医薬品名"].map(name2firm)
data[data["メーカー名"].isna()].shape

(31, 132)

In [ ]:
os.chdir("C:/Users/hakos/GDrive/Downloads/Research/data/yakka")
# data.to_csv('../merged/len3_ndb_blp_DN_firm_filled.csv',encoding='utf-8')